<a href="https://colab.research.google.com/github/BritneyMuller/colab-notebooks/blob/master/%F0%9F%90%B8_SF_Internal_Link_Opportunities_%F0%9F%90%B8_%5BMake_a_Copy!%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Automate Internal Link Opportunities [Workflow] from Everett Sizemore's brilliant tutorial,
#  "How I Found Internal Linking Opportunities With Vector Embeddings": https://moz.com/blog/internal-linking-opportunities-with-vector-embeddings
#  Dev: Britney Muller
#  Internal Link Analysis: Link Cleaning, Embeddings Processing, and Relationship Discovery
#  This script performs:
#  1. Raw all_inlinks cleaning
#  2. Raw vector embeddings preprocessing and cleaning
#  3. Finds contextually related pages using vector embeddings
#  4. Analyzes + identifies top link opportunities between related pages

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files
import re
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment
import tempfile

# ----------- PART 1: LINK DATASET CLEANING -----------

def clean_link_dataset(df):
    """
    Clean and process link dataset according to specified rules.

    Parameters:
    df (pandas.DataFrame): Input DataFrame containing link data

    Returns:
    pandas.DataFrame: Cleaned DataFrame with Source, Destination, and Anchor columns
    """
    # Make a copy to avoid modifying the original
    df = df.copy()

    print("Initial shape:", df.shape)

    # 1. Sort by Type and filter for Hyperlinks
    print("\nSorting by Type and filtering for Hyperlinks...")
    if 'Type' in df.columns:
        df = df.sort_values('Type')
        df = df[df['Type'] == 'Hyperlink'].drop('Type', axis=1)
        print("Shape after Type filtering:", df.shape)

    # 2. Sort by Status Code and filter for 200
    print("\nSorting by Status Code and filtering for 200 status...")
    if 'Status Code' in df.columns:
        df = df.sort_values('Status Code')
        df = df[df['Status Code'] == 200]
        columns_to_drop = ['Status Code', 'Status'] if 'Status' in df.columns else ['Status Code']
        df = df.drop(columns_to_drop, axis=1)
        print("Shape after Status filtering:", df.shape)

    # 3. Delete specified columns if they exist
    columns_to_drop = [
        'Size (Bytes)', 'Follow', 'Target', 'Rel',
        'Path Type', 'Link Path', 'Link Origin'
    ]
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]

    if columns_to_drop:
        print("\nRemoving unnecessary columns...")
        df = df.drop(columns_to_drop, axis=1)
    print("Remaining columns:", df.columns.tolist())

    # 4. Sort by Link Position
    if 'Link Position' in df.columns:
        print("\nSorting by Link Position...")
        df = df.sort_values('Link Position')

        # 5. Filter for Content and Aside in Link Position
        print("\nFiltering for Content and Aside positions...")
        df = df[df['Link Position'].isin(['Content', 'Aside'])]
        print("Shape after Link Position filtering:", df.shape)

    # 6. Sort by Source and clean Source URLs
    print("\nSorting and cleaning Source URLs...")
    source_col = 'Source' if 'Source' in df.columns else df.columns[0]
    df = df.sort_values(source_col)

    def is_valid_page(url):
        if pd.isna(url):
            return False

        invalid_patterns = [
            # ***MODIFY THE BELOW URL PATTERNS IF THERE ARE CERTAIN STRUCTURES YOU WANT TO INCLUDE/EXCLUDE***
            'category/', 'tag/', 'sitemap', 'search', '/home/', 'index'
        ]
        return not any(pattern in str(url).lower() for pattern in invalid_patterns)

    df = df[df[source_col].apply(is_valid_page)]
    print(f"Shape after {source_col} URL cleaning:", df.shape)

    # 7. Sort by Destination and clean Destination URLs
    print("\nSorting and cleaning Destination URLs...")
    dest_col = 'Destination' if 'Destination' in df.columns else df.columns[1]
    df = df.sort_values(dest_col)
    df = df[df[dest_col].apply(is_valid_page)]
    print(f"Shape after {dest_col} URL cleaning:", df.shape)

    # 8. Sort by Alt Text (Z to A) and process Alt Text if it exists
    if 'Alt Text' in df.columns and 'Anchor' in df.columns:
        print("\nSorting by Alt Text and processing...")
        df = df.sort_values('Alt Text', ascending=False)
        df.loc[df['Alt Text'].notna(), 'Anchor'] = df['Alt Text']
        df = df.drop('Alt Text', axis=1)

    # 9. Handle self-linking URLs
    print("\nProcessing self-linking URLs...")
    # Create the links to self column
    df['links to self'] = np.where(df[source_col] == df[dest_col], 'Match', 'No Match')

    # Sort by links to self column A-Z
    df = df.sort_values('links to self')

    # Remove matching rows and the column
    df = df[df['links to self'] != 'Match']
    df = df.drop('links to self', axis=1)
    print("Shape after removing self-links:", df.shape)

    # Remove Link Position column if it exists and is no longer needed
    if 'Link Position' in df.columns:
        df = df.drop('Link Position', axis=1)

    # Ensure we have the expected columns or rename as needed
    if source_col != 'Source' or dest_col != 'Destination':
        df = df.rename(columns={source_col: 'Source', dest_col: 'Destination'})

    # If Anchor doesn't exist, add an empty one
    if 'Anchor' not in df.columns:
        df['Anchor'] = ''

    # Final column ordering
    final_columns = ['Source', 'Destination', 'Anchor']
    other_columns = [col for col in df.columns if col not in final_columns]
    df = df[final_columns + other_columns]

    return df

# ----------- PART 2: EMBEDDINGS PREPROCESSING -----------

def clean_embeddings_data(df):
    """
    Clean and preprocess embeddings data according to specified rules.

    Parameters:
    df (pandas.DataFrame): Input DataFrame containing embeddings data

    Returns:
    pandas.DataFrame: Cleaned DataFrame with URL and Embeddings columns
    """
    print("\n--- CLEANING EMBEDDINGS DATA ---")

    # Make a copy to avoid modifying the original
    df = df.copy()

    print("Initial shape:", df.shape)

    # Find the embeddings column - usually it contains "embeddings" or "Extract"
    embeddings_col = None
    for col in df.columns:
        if 'embeddings' in col.lower() or 'extract' in col.lower():
            embeddings_col = col
            break

    if not embeddings_col:
        raise ValueError("Could not find a column containing embeddings data. Please ensure your CSV has a column with 'embeddings' or 'extract' in its name.")

    # Sort the embeddings column from Z to A
    print(f"\nSorting {embeddings_col} column from Z to A...")
    df = df.sort_values(embeddings_col, ascending=False)

    # Delete any row where the embeddings column is not a string of numbers
    print("\nRemoving rows with invalid embeddings data...")

    # Function to check if a string contains numbers, commas, dots, and basic formatting
    def is_valid_embedding(text):
        if pd.isna(text):
            return False

        # Check if it contains words like "timeout", "error", etc.
        invalid_words = ['timeout', 'error', 'null', 'undefined', 'nan']
        if any(word in str(text).lower() for word in invalid_words):
            return False

        # Check if it looks like a vector (contains numbers and separators)
        text_str = str(text)
        has_numbers = any(c.isdigit() for c in text_str)
        has_separators = ',' in text_str or '.' in text_str
        return has_numbers and has_separators

    df = df[df[embeddings_col].apply(is_valid_embedding)]
    print("Shape after removing invalid embeddings:", df.shape)

    # Verify all URLs have a status code 200
    if 'Status Code' in df.columns:
        print("\nFiltering for status code 200...")
        df = df[df['Status Code'] == 200]
        print("Shape after status code filtering:", df.shape)

    # Determine the URL column
    url_col = None
    potential_url_cols = ['URL', 'Address', 'Url', 'address']

    for col in potential_url_cols:
        if col in df.columns:
            url_col = col
            break

    if not url_col:
        # Assume the first column that's not the embeddings column is the URL column
        for col in df.columns:
            if col != embeddings_col and col != 'Status Code' and col != 'Status':
                url_col = col
                break

    if not url_col:
        raise ValueError("Could not identify a URL column. Please ensure your CSV has a column with URLs.")

    # Delete the Status Code and Status columns if they exist
    cols_to_drop = [col for col in ['Status Code', 'Status'] if col in df.columns]
    if cols_to_drop:
        df = df.drop(cols_to_drop, axis=1)

    # Create a new DataFrame with just the URL and Embeddings columns, properly renamed
    cleaned_df = pd.DataFrame()
    cleaned_df['URL'] = df[url_col]
    cleaned_df['Embeddings'] = df[embeddings_col]

    print("\nFinal embeddings data shape:", cleaned_df.shape)
    print("Final columns:", cleaned_df.columns.tolist())

    return cleaned_df

# ----------- PART 3: URL RELATIONSHIP ANALYSIS -----------

# Function to upload a file with a specific name
def upload_file(prompt_message):
    print(prompt_message)
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No file was uploaded.")
    file_name = list(uploaded.keys())[0]
    print(f"Uploaded: {file_name}")
    return file_name

# Function to find top N related pages based on cosine similarity
def find_related_pages(df, top_n=5):
    print(f"Finding top {top_n} related pages for each URL...")
    related_pages = {}
    embeddings = np.stack(df['Embeddings'].values)
    urls = df['URL'].values

    # Calculate cosine similarity matrix
    cosine_similarities = cosine_similarity(embeddings)

    # For each URL, find the most similar URLs
    for idx, url in enumerate(urls):
        # Get the indices of the top_n most similar URLs, excluding itself
        similar_indices = cosine_similarities[idx].argsort()[-(top_n+1):][::-1]
        # Filter out the URL itself
        similar_indices = [i for i in similar_indices if urls[i] != url][:top_n]
        # Get the related URLs
        related_urls = urls[similar_indices].tolist()
        related_pages[url] = related_urls

    return related_pages

# ----------- MAIN PROCESS -----------

print("=" * 80)
print("Internal Linking Analysis Tool")
print("Combining link cleaning, embeddings analysis, and relationship discovery")
print("=" * 80)

# Step 1: Start with raw link data and clean it
print("\n--- STEP 1: CLEANING ALL_INLINKS DATA ---")

# Upload raw link export
link_export_file = upload_file("Please upload your raw all_inlinks export CSV file...")

# Read the link export file
df_links = pd.read_csv(link_export_file)
print("\nLink export file information:")
print(f"Columns: {df_links.columns.tolist()}")
print(f"Shape: {df_links.shape}")

# Clean the link dataset
print("\nCleaning link dataset...")
df_cleaned_links = clean_link_dataset(df_links)

# Save the cleaned links to a CSV (for internal use only)
cleaned_links_file = 'cleaned_links.csv'
df_cleaned_links.to_csv(cleaned_links_file, index=False)
print(f"\nCleaned link data saved to {cleaned_links_file} (for internal use)")


# Step 2: Process embeddings data
print("\n--- STEP 2: EMBEDDINGS PROCESSING ---")

# Upload embeddings file
embeddings_file = upload_file("Please upload your raw embeddings CSV export...")

# Read the embeddings file
df_embeddings_raw = pd.read_csv(embeddings_file)
print(f"Embeddings file columns: {df_embeddings_raw.columns.tolist()}")

# Clean the embeddings data
df_embeddings = clean_embeddings_data(df_embeddings_raw)

# Save the cleaned embeddings to a CSV (for internal use only)
cleaned_embeddings_file = 'cleaned_embeddings.csv'
df_embeddings.to_csv(cleaned_embeddings_file, index=False)
print(f"\nCleaned embeddings data saved to {cleaned_embeddings_file} (for internal use)")


# Step 3: Convert embeddings from string to numpy arrays
print("\n--- STEP 3: ANALYZING RELATIONSHIPS BETWEEN URLS ---")
print("Converting embeddings to arrays...")
df_embeddings['Embeddings'] = df_embeddings['Embeddings'].apply(
    lambda x: np.array([float(i) for i in x.strip('[]').replace("'", "").split(',')])
)

# Find related pages
related_pages = find_related_pages(df_embeddings, top_n=5)

# Create a dictionary to store all inlinks for each URL
print("Processing inlinks data...")
inlinks_dict = {}
for target_url in df_embeddings['URL']:
    # Find all source URLs that link to this target URL
    linking_sources = df_cleaned_links[df_cleaned_links['Destination'] == target_url]['Source'].tolist()
    # Join them with commas
    inlinks_dict[target_url] = ', '.join(linking_sources) if linking_sources else ''

# Create the final DataFrame
print("Creating internal linking opportunities DataFrame...")
output_data = []

for url, related_urls in related_pages.items():
    # Pad the related_urls list to always have 5 items
    padded_related_urls = related_urls + [None] * (5 - len(related_urls))

    # Get the list of URLs that link to this target URL
    links_to_target = inlinks_dict.get(url, '')

    # Modify here: Replace empty values with "none"
    if not links_to_target:
        links_to_target = "none"

    # Create a row for this URL
    row = {
        'Target URL': url,
        'Links to Target URL': links_to_target
    }

    # Add related URLs and check if they link to the target URL
    for i, related_url in enumerate(padded_related_urls, 1):
        row[f'Related URL {i}'] = related_url
        if related_url is not None:
            exists_status = "Exists" if related_url in df_cleaned_links[df_cleaned_links['Destination'] == url]['Source'].values else "Not Found"
            row[f'URL {i} links to A?'] = exists_status
        else:
            row[f'URL {i} links to A?'] = "Not Found"

    output_data.append(row)

# Create the final DataFrame with columns in the correct order
column_order = ['Target URL', 'Links to Target URL']
for i in range(1, 6):
    column_order.extend([f'Related URL {i}', f'URL {i} links to A?'])

final_df = pd.DataFrame(output_data)
final_df = final_df[column_order]

# Save to Excel file with formatting
output_file_name = 'internal_link_opportunities.xlsx'
print(f"Creating Excel file with formatting: {output_file_name}...")

# Create a temporary file
with tempfile.NamedTemporaryFile(suffix='.xlsx') as tmp:
    # Save DataFrame to Excel
    with pd.ExcelWriter(tmp.name, engine='openpyxl') as writer:
        final_df.to_excel(writer, index=False, sheet_name='URL Analysis')

        # Get the workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets['URL Analysis']

        # Define fills for "Exists" and "Not Found"
        green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # Light green
        red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")    # Light red

        # Apply conditional formatting to all "URL X links to A?" columns
        for col_idx in [4, 6, 8, 10, 12]:  # Columns D, F, H, J, L in Excel (1-indexed)
            for row_idx in range(2, len(final_df) + 2):  # Starting from row 2 (skipping header)
                cell = worksheet.cell(row=row_idx, column=col_idx)
                if cell.value == "Exists":
                    cell.fill = green_fill
                elif cell.value == "Not Found":
                    cell.fill = red_fill

        # Adjust column widths and formatting for better readability
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter
            for cell in col:
                # Set alignment for all cells
                cell.alignment = Alignment(vertical='center')

                # For "Links to Target URL" column, wrap text
                if column == 'B':
                    cell.alignment = Alignment(vertical='center', wrap_text=True)

                # Calculate max length for column width
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))

            # Set column widths
            if column == 'B':  # "Links to Target URL" column
                worksheet.column_dimensions[column].width = 50  # Fixed width
            else:
                adjusted_width = max(max_length, 12)  # Minimum width of 12
                worksheet.column_dimensions[column].width = min(adjusted_width, 40)  # Maximum width of 40

        # Freeze the top row
        worksheet.freeze_panes = 'A2'

    # Read the temporary file
    with open(tmp.name, 'rb') as f:
        excel_data = f.read()

    # Write to a file in Colab
    with open(output_file_name, 'wb') as f:
        f.write(excel_data)

# Download the Excel file
files.download(output_file_name)

# Also save a CSV version for compatibility
csv_file_name = 'internal_link_opportunities.csv'
print(f"Also saving a plain CSV version: {csv_file_name}")
final_df.to_csv(csv_file_name, index=False)

# Only download the final analysis files
print("\nDownloading internal link opportunities files...")
files.download(output_file_name)  # Excel file with formatting
files.download(csv_file_name)     # CSV version

print("\n=== Process completed successfully! ===")
print("The internal link opportunities contains all the information you need:")
print(f"1. {output_file_name} - Excel file with color formatting")
print(f"2. {csv_file_name} - CSV file for compatibility with other tools")
print("\nThese files show: ")
print("* Your target URLs")
print("* All URLs linking to each target")
print("* The top 5 contextually similar pages for each target")
print("* Whether each related page links to the target (highlighted)")

Internal Linking Analysis Tool
Combining link cleaning, embeddings analysis, and relationship discovery

--- STEP 1: CLEANING ALL_INLINKS DATA ---
Please upload your raw all_inlinks export CSV file...


Saving all_inlinks.csv to all_inlinks.csv
Uploaded: all_inlinks.csv

Link export file information:
Columns: ['Type', 'Source', 'Destination', 'Size (Bytes)', 'Alt Text', 'Anchor', 'Status Code', 'Status', 'Follow', 'Target', 'Rel', 'Path Type', 'Link Path', 'Link Position', 'Link Origin']
Shape: (6419, 15)

Cleaning link dataset...
Initial shape: (6419, 15)

Sorting by Type and filtering for Hyperlinks...
Shape after Type filtering: (5948, 14)

Sorting by Status Code and filtering for 200 status...
Shape after Status filtering: (5940, 12)

Removing unnecessary columns...
Remaining columns: ['Source', 'Destination', 'Alt Text', 'Anchor', 'Link Position']

Sorting by Link Position...

Filtering for Content and Aside positions...
Shape after Link Position filtering: (1092, 5)

Sorting and cleaning Source URLs...
Shape after Source URL cleaning: (1092, 5)

Sorting and cleaning Destination URLs...
Shape after Destination URL cleaning: (1092, 5)

Sorting by Alt Text and processing...

Proces

Saving embeddings.csv to embeddings.csv
Uploaded: embeddings.csv
Embeddings file columns: ['Address', 'Content Type', 'Status Code', 'Status', '(ChatGPT) Extract embeddings from page content 1']

--- CLEANING EMBEDDINGS DATA ---
Initial shape: (85, 5)

Sorting (ChatGPT) Extract embeddings from page content 1 column from Z to A...

Removing rows with invalid embeddings data...
Shape after removing invalid embeddings: (81, 5)

Filtering for status code 200...
Shape after status code filtering: (81, 5)

Final embeddings data shape: (81, 2)
Final columns: ['URL', 'Embeddings']

Cleaned embeddings data saved to cleaned_embeddings.csv (for internal use)

--- STEP 3: ANALYZING RELATIONSHIPS BETWEEN URLS ---
Converting embeddings to arrays...
Finding top 5 related pages for each URL...
Processing inlinks data...
Creating internal linking opportunities DataFrame...
Creating Excel file with formatting: internal_link_opportunities.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


=== Process completed successfully! ===
The internal link opportunities contains all the information you need:
1. internal_link_opportunities.xlsx - Excel file with color formatting
2. internal_link_opportunities.csv - CSV file for compatibility with other tools

These files show: 
* Your target URLs
* All URLs linking to each target
* The top 5 contextually similar pages for each target
* Whether each related page links to the target (highlighted)


# Checkout the [Actionable AI For Marketers Course](https://maven.com/britney-muller/actionable-ai-for-marketers)🚀🚀🚀 For More Automated Workflows like this one!